In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2024-09-09 14:02:33.230115: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-09 14:02:33.490821: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
histograms = pd.read_csv("histogram_matrix.csv")
#histograms = histograms.set_index('sample_num')
clinical = pd.read_csv("clinical_binary.csv")
#clinical = clinical.set_index('sample_num')

FileNotFoundError: [Errno 2] No such file or directory: 'clinical_binary.csv'

In [ ]:
for i in range (1, 193):
    if i not in histograms['sample_num'].values:
        print(i)

In [ ]:
((134 - 96)) #todo: get UQ sample 38 data

In [ ]:
histograms.head()

In [ ]:
clinical.head()

# {"ctrl":0, "als":1}
# {"Male":0, "Female":1}
# {"ucsf":0, "uq":1}
# {"white":0, "not white":1}

In [ ]:
merged_df = pd.merge(histograms, clinical, on='sample_num')

In [ ]:
merged_df.drop(columns = ['sample_num'], inplace=True)

In [ ]:
merged_df_clean = merged_df.dropna() #drop rows with missing age values

In [ ]:
merged_df.describe()

In [ ]:
def train(X,Y,X_shape):
    
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=0)

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(8, input_shape=(X_shape,), activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid')) #gives binary output

    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=100,  validation_split=0.2, batch_size=5, verbose=1)

    plt.plot(history.history["accuracy"], label="train")
    plt.plot(history.history["val_accuracy"], label="validation")
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc='upper left')
    plt.show()

## combined cohorts

In [ ]:
Y = merged_df_clean['type_binary']
X = merged_df_clean[merged_df_clean.columns.difference(['type_binary'])]
#X = merged_df

In [ ]:
print(X.isnull().sum())  # Check for NaNs in the features
print(Y.isnull().sum())  # Check for NaNs in the labels

In [ ]:
train(X,Y,355)

## ucsf

In [ ]:
ucsf = merged_df_clean[merged_df_clean['cohort_binary'] == 0]
uq = merged_df_clean[merged_df_clean['cohort_binary'] == 1]

In [ ]:
ucsf

In [ ]:
X = ucsf[ucsf.columns.difference(['type_binary', 'cohort_binary'])]
Y = ucsf['type_binary']

In [ ]:
train(X,Y,354)

## uq

In [ ]:
uq

In [ ]:
X = uq[uq.columns.difference(['type_binary', 'cohort_binary'])]
Y = uq['type_binary']

In [ ]:
train(X,Y,354)